In [1]:
from docx import Document
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import RGBColor
from docx.shared import Cm
from docx.enum.table import WD_ROW_HEIGHT_RULE
import pickle

In [2]:
def read_data(file = 'parsed_data_2.pi'):
    
    with open(file, 'rb') as f:
        return pickle.load(f)

In [3]:
def orange_cell(cell):
    
    # Set a cell background (shading) color to RGB D9D9D9.
    shading_elm = parse_xml(r'<w:shd {} w:fill="#FF7F50"/>'.format(nsdecls('w')))
    cell._tc.get_or_add_tcPr().append(shading_elm)

In [4]:
def write_white(cell, text):
    
    if text is None:
        text = ''
    if type(text) is not str:
        text = str(text)
    
    text = cell.paragraphs[0].add_run(text)
    white = RGBColor(255,255,255)
    text.font.color.rgb = white

In [5]:
def white_on_orange(cell, text):
    
    orange_cell(cell)
    write_white(cell, text)

In [6]:
def white_row(table, row_num, max_cell):
    
    merge_cells(table, row_num, max_cell)
    row = table.rows[row_num]
    row.height_rule = WD_ROW_HEIGHT_RULE.EXACTLY
    row.height = Cm(.3)

In [7]:
def add_table(doc, num_rows, num_cols):
    
    table = doc.add_table(num_rows,num_cols)
    table.style = 'Table Grid'
    
    return table

In [8]:
def merge_cells(table, row, max_cell, start_cell=0):
    
    merge_anchor = table.cell(row, start_cell)
    for i in range(start_cell+1, max_cell+1):
        merge_anchor.merge(table.cell(row, i))
    
    return merge_anchor

In [9]:
def write_cell(table, row, col, text):
    
    if text is None:
        text = ''
    
    if type(text) is not str:
        text = str(text)
    
    table.cell(row, col).text = text

In [10]:
def header(doc, dados):
    
    header = dados['header']
    
    table = add_table(doc, 2,2)
    white_on_orange(table.cell(0,0), header['numero_meta'])
    write_cell(table, 0, 1, header['desc_meta'])
    white_row(table, 1, 1)

In [11]:
def vinculacao_pdm(doc, dados):
    
    vinculos = dados['vinculacao_pdm']
    #uma para cada vinculo, uma para header e outra para espaço
    num_rows = len(vinculos)+2
    table = add_table(doc, num_rows, 2)
    
    header = merge_cells(table, 0, 1)
    white_on_orange(header, 'Vinculação com o Programa de Metas 2021-2024')
    
    for i, row in enumerate(vinculos):
        
        write_cell(table, i+1,0, row['meta_pdm'])
        write_cell(table, i+1,1, row['desc_meta_pdm'])

    white_row(table, num_rows-1, 1)

In [12]:
def vinculacao_ppa(doc, dados):
    
    vinculos = dados['vinculacao_ppa']
    #uma para cada vinculo, uma para header e outra para espaço
    num_rows = len(vinculos)+2
    table = add_table(doc, num_rows, 2)
    
    header = merge_cells(table, 0, 1)
    white_on_orange(header, 'Vinculação com o Plano Plurianual 2022-2025')
    
    for i, row in enumerate(vinculos):
        
        write_cell(table, i+1,0, row['vinculo_ppa'])
        write_cell(table, i+1,1, row['descricao_vinculo_ppa'])
    
    white_row(table, num_rows-1, 1)

In [13]:
def indicadores_priorizados(doc, dados):
    
    itens = dados['indicadores_priorizados']
    #uma para cada item, duas para header e outra para espaço
    num_rows = len(itens)+3
    table = add_table(doc, num_rows, 4)
    
    header = merge_cells(table, 0, 3)
    white_on_orange(header, 'Indicadores da Agenda Municipal 2030 priorizados')
    
    white_on_orange(table.cell(1,0), 'Número')
    white_on_orange(table.cell(1,1), 'Redação')
    white_on_orange(table.cell(1,2), 'Meta 2030')
    white_on_orange(table.cell(1,3), 'Previsão 2021-2024')

    
    for i, row in enumerate(itens):
        
        row_num = i+2
        write_cell(table, row_num,0, row['numero_indicador'])
        write_cell(table, row_num,1, row['redacao'])
        write_cell(table, row_num,2, row['meta_2030'])
        write_cell(table, row_num,3, row['previsao_21_24'])

    white_row(table, num_rows-1, 3)

In [14]:
def acoes(doc, dados):
    
    itens = dados['acoes']
    #uma para cada item, duas para header
    num_rows = len(itens)+1
    table = add_table(doc, num_rows, 4)
    
    header = merge_cells(table, 0, 1)
    white_on_orange(header, 'Ação')
    white_on_orange(table.cell(0,2), 'Orgão Responsável')
    white_on_orange(table.cell(0,3), 'Marco de atingimento')
    
    for i, row in enumerate(itens):
        
        row_num = i+1
        
        write_cell(table, row_num,0, row['letra_acao'])
        write_cell(table, row_num,1, row['desc_acao'])
        write_cell(table, row_num,2, row['orgao_responsavel'])
        write_cell(table, row_num,3, row['marco_de_atingimento'])


In [15]:
def write_sheet(doc, sheet_data):
    
    header(doc, sheet_data)
    vinculacao_pdm(doc, sheet_data)
    vinculacao_ppa(doc, sheet_data)
    indicadores_priorizados(doc, sheet_data)
    acoes(doc, sheet_data)
    
    doc.add_page_break()

In [16]:
data = read_data()

In [17]:
dados = data[0][0]

In [18]:
doc = Document()

In [19]:
for planilha in data:
    for sheet in planilha:
        write_sheet(doc, sheet)

In [20]:
doc.save('word_ods_gabi_final.docx')

In [29]:
with open('word_ods_gabi_final.docx', 'rb') as f:
    texto = f.read()

In [39]:
from zipfile import ZipFile
from io import BytesIO

In [41]:
zipfile = ZipFile(BytesIO(texto))

In [45]:
zipfile.extractall()